# Football World Cup 2022

In [1]:
import pandas as pd
from scipy.stats import poisson

In [2]:
dataGroup = pd.read_html("https://www.wallchart.io/football/world-cup-2022.html")
datamatches = pd.read_csv("/Users/hilmanyusoh/Documents/PWFWCup 2022/clean_fifa_worldcup_matches.csv")
datafixture = pd.read_csv("/Users/hilmanyusoh/Documents/PWFWCup 2022/clean_fifa_worldcup_fixture.csv")

# 1.Calculate team strenght

In [3]:
# split dataframe into datahome and dataway
data_home = datamatches[["HomeTeam","HomeGoals","AwayGoals"]]
data_away = datamatches[["AwayTeam","HomeGoals","AwayGoals"]]

In [4]:
# rename columns
data_home = data_home.rename(columns={"HomeTeam": "Team", "HomeGoals": "GoalsScored", "AwayGoals": "GoalsConceded"})
data_away = data_away.rename(columns={"AwayTeam": "Team", "HomeGoals": "GoalsConceded", "AwayGoals": "GaolsScored"})
                         

In [5]:
# concat data_home and data_away, group by team and calculate the mean
df_team_strenght = pd.concat([data_home, data_away], ignore_index=True).groupby('Team').mean()
df_team_strenght

,GoalsScored,GoalsConceded,GaolsScored
Team,,,
Algeria,0.833333,1.461538,1.142857
Angola,0.000000,0.666667,0.500000
Argentina,1.913793,1.148148,1.130435
Australia,1.000000,1.937500,0.666667
Austria,2.285714,1.620690,0.733333
...,...,...,...
Uruguay,2.060606,1.321429,0.826087
Wales,2.000000,0.800000,0.500000
West Germany,2.500000,1.241935,1.500000


## 2 Function predict_point

In [7]:
def predict_point(Home,Away):
    if home in df_Team_strenght.index and away in df_team_strenght.index:
        # Goals_scored * goals_conceded
        touch_home = df_team_strenght[home,'GoalsScored'] * df_team_strenght.at[away,'GoalsConceded']
        touch_away = df_team_strenght[home,'GoalsScored'] * df_team_strenght.at[home,'GoalsConcedde']
        prob_home, prob_away, = 0, 0, 0
        for x in range(0,11): #numeber of goals home team
            for y in range(0,11): #number of goals away team
                p = poisson.pmf(x,touch_home) * poisson.pmf(y,touch_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_draw += p
                else:
                    prob_away += p
                    
        pionts_home = 3 * prob_home + prob_draw
        pionts_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 8)
                

    